In [3]:
# importing the required modules. 
import random 
import json 
import pickle 
import numpy as np 
import nltk 
  
from keras.models import Sequential 
from nltk.stem import WordNetLemmatizer 
from keras.layers import Dense, Activation, Dropout 
from keras.optimizers import SGD 
  
lemmatizer = WordNetLemmatizer() 

In [4]:
# reading the json.intense file 
intents = json.loads(open("intents/intents.json").read()) 
  
# creating empty lists to store data 
words = [] 
classes = [] 
documents = [] 
ignore_letters = ["?", "!", ".", ","] 
for intent in intents['intents']: 
    for pattern in intent['patterns']: 
        # separating words from patterns 
        word_list = nltk.word_tokenize(pattern) 
        words.extend(word_list)  # and adding them to words list 
          
        # associating patterns with respective tags 
        documents.append(((word_list), intent['tag'])) 
  
        # appending the tags to the class list 
        if intent['tag'] not in classes: 
            classes.append(intent['tag']) 
  
# storing the root words or lemma 
words = [lemmatizer.lemmatize(word) 
         for word in words if word not in ignore_letters] 
words = sorted(set(words)) 
  
# saving the words and classes list to binary files 
pickle.dump(words, open('words.pkl', 'wb')) 
pickle.dump(classes, open('classes.pkl', 'wb')) 

In [7]:
# we need numerical values of the 
# words because a neural network 
# needs numerical values to work with 
training = [] 
output_empty = [0]*len(classes) 
for document in documents: 
    bag = [] 
    word_patterns = document[0] 
    word_patterns = [lemmatizer.lemmatize( 
        word.lower()) for word in word_patterns] 
    for word in words: 
        bag.append(1) if word in word_patterns else bag.append(0) 
          
    # making a copy of the output_empty 
    output_row = list(output_empty) 
    output_row[classes.index(document[1])] = 1
    training.append((np.array(bag), np.array(output_row))) 
random.shuffle(training) 
#training = np.array(training) 
  
# splitting the data 
# train_x = list(training[:, 0]) 
# train_y = list(training[:, 1]) 
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

In [8]:
# training_data = []
# output_empty = [0]*len(classes) 
# for document in documents:
#     bag = []
#     word_patterns = document[0]
#     word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
#     for word in words:
#         bag.append(1) if word in word_patterns else bag.append(0)
          
#     output_row = list(output_empty)
#     output_row[classes.index(document[1])] = 1
    
#     training_data.append((np.array(bag), np.array(output_row)))

# random.shuffle(training_data)

# train_x = np.array([item[0] for item in training_data])
# train_y = np.array([item[1] for item in training_data])

In [9]:
# creating a Sequential machine learning model 
model = Sequential() 
model.add(Dense(128, input_shape=(len(train_x[0]), ), 
                activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(len(train_y[0]),  
                activation='softmax')) 


# compiling the model 
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, metrics=['accuracy']) 
hist = model.fit(np.array(train_x), np.array(train_y), 
                 epochs=200, batch_size=5, verbose=1) 

# compiling the model 
# sgd = SGD(lr=0.01, momentum=0.9, nesterov=True) 
# model.compile(loss='categorical_crossentropy', 
#               optimizer=sgd, metrics=['accuracy']) 
# hist = model.fit(np.array(train_x), np.array(train_y), 
#                  epochs=200, batch_size=5, verbose=1) 
  
# saving the model 
model.save("chatbotmodel.h5", hist) 
  
# print statement to show the  
# successful training of the Chatbot model 
print("Yay!") 

Epoch 1/200


C:\Users\av945\anaconda3\envs\chatbot\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 3.4295
Epoch 2/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0382 - loss: 3.3676     
Epoch 3/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0558 - loss: 3.3179     
Epoch 4/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1730 - loss: 3.2653 
Epoch 5/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.1076 - loss: 3.2186
Epoch 6/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - accuracy: 0.1643 - loss: 3.1767
Epoch 7/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - accuracy: 0.1060 - loss: 3.1304
Epoch 8/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - accuracy: 0.1763 - loss: 3.0002
Epoch 9/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - accuracy: 0.1743 - loss: 2.9963
Epoch 10/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - accuracy: 0.2476 - loss: 2.8351
Epoch 11/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.2859 - loss: 2.6387
Epoch 12/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 875

Yay!
